# ResNet 50 (Transfer Learning)

In [2]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchsummary import summary
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from ptflops import get_model_complexity_info
import time

# Import custom utility module
from utils import get_transform, prepare_train_test_data, calculate_evaluation_metrics

In [3]:
# Device configuration (Select CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n Device : {device}")


 Device : cuda


### Load and prepare the data

In [4]:
# Transform data
image_input_size = (3, 224, 224) # Image input size for the model
transform = get_transform(resize_image_size = (224, 224))

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

# Model keyword
model_keyword = "resnet50"

### Load pre-trained ResNet50 model and modify it

In [5]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training

# Print summary
summary(model, input_size=image_input_size)

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### Train the model

In [35]:
# Initialize a df to log training/validation metrics
log_df = []

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    
    # Adjust learning rate
    scheduler.step(val_loss)
    
    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, image_input_size, as_strings=False, print_per_layer_stat=False)
    
    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df.append(new_row)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"FLOPs: {(flops / 1e9 ):.2f} GFLOPs")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"model_outputs_data/best_{model_keyword}_model.pth")

# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df = pd.DataFrame(log_df)
log_df.to_csv(f"model_outputs_data/model_evaluation_logs/training_{model_keyword}.csv", index=False)

C:\Users\ishar\AppData\Local\Temp\ipykernel_20404\3796617994.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)


Epoch [1/50], Train Loss: 0.5716, Val Loss: 0.4982, Val Accuracy: 81.50%, FLOPs: 4.13 GFLOPs
Epoch [2/50], Train Loss: 0.4450, Val Loss: 0.5995, Val Accuracy: 73.72%, FLOPs: 4.13 GFLOPs
Epoch [3/50], Train Loss: 0.4120, Val Loss: 0.4799, Val Accuracy: 77.97%, FLOPs: 4.13 GFLOPs
Epoch [4/50], Train Loss: 0.3880, Val Loss: 0.4515, Val Accuracy: 81.79%, FLOPs: 4.13 GFLOPs
Epoch [5/50], Train Loss: 0.3660, Val Loss: 0.3950, Val Accuracy: 83.26%, FLOPs: 4.13 GFLOPs
Epoch [6/50], Train Loss: 0.3216, Val Loss: 0.3371, Val Accuracy: 87.22%, FLOPs: 4.13 GFLOPs
Epoch [7/50], Train Loss: 0.3226, Val Loss: 0.3376, Val Accuracy: 83.85%, FLOPs: 4.13 GFLOPs
Epoch [8/50], Train Loss: 0.2850, Val Loss: 0.3357, Val Accuracy: 87.37%, FLOPs: 4.13 GFLOPs
Epoch [9/50], Train Loss: 0.2566, Val Loss: 0.3092, Val Accuracy: 86.93%, FLOPs: 4.13 GFLOPs
Epoch [10/50], Train Loss: 0.2566, Val Loss: 0.3791, Val Accuracy: 83.11%, FLOPs: 4.13 GFLOPs
Epoch [11/50], Train Loss: 0.1966, Val Loss: 0.5201, Val Accuracy: 81

### Prediction and Model evaluation

In [36]:
# Test the model
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Print evaluation metrics
calculate_evaluation_metrics(all_labels, all_preds, model_keyword)


Test Accuracy: 87.32%
Precision: 0.91
Recall: 0.83
F1-Score: 0.87

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.84      0.92      0.88       719
   Internal Waves       0.91      0.83      0.87       740

         accuracy                           0.87      1459
        macro avg       0.88      0.87      0.87      1459
     weighted avg       0.88      0.87      0.87      1459


Confusion Matrix:
[[659  60]
 [125 615]]


In [40]:
# Save prediction data to a CSV file (As a backup)
results_df = pd.DataFrame({
    "true_labels": all_labels,
    "predicted_labels": all_preds
})
results_df.to_csv("model_outputs_data/model_prediction_logs/resnet50_labels_predictions.csv", index=False)